In [1]:
import tensorflow as tf
import numpy as np
import os
import sys

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
input_vocab_size = 30000
output_vocab_size = 30005
input_ph = tf.placeholder(dtype=tf.float32, shape=[None, None, input_vocab_size])
output_ph = tf.placeholder(dtype=tf.float32, shape=[None, None, output_vocab_size])

In [3]:
batch_size = tf.cast(tf.shape(input_ph)[0], tf.int32)
seq_length = tf.cast(tf.shape(input_ph)[1], tf.int32)

In [6]:
input_embedding_size = 650
W_input_embedding = tf.Variable(tf.random_normal([input_vocab_size, input_embedding_size], stddev=0.001))
b_input_embedding = tf.Variable(tf.random_normal([input_embedding_size], stddev=0.001))

In [14]:
encoder_gru_size = 1024
encoder_gru_fw = tf.nn.rnn_cell.GRUCell(encoder_gru_size)
encoder_gru_bw = tf.nn.rnn_cell.GRUCell(encoder_gru_size)

In [13]:
decoder_gru_size = 1152
decoder_gru = tf.nn.rnn_cell.GRUCell(decoder_gru_size)

In [22]:
input_2d = tf.reshape(input_ph, [-1, input_vocab_size])
embedded_input_2d = tf.nn.relu(tf.matmul(input_2d, W_input_embedding) + b_input_embedding)
embedded_input_3d = tf.reshape(embedded_input_2d, [batch_size, seq_length, input_vocab_size])

In [24]:
# [h_fw, h_bw]
bidirect_encoder_states, _ = tf.nn.bidirectional_dynamic_rnn(encoder_gru_fw, encoder_gru_bw, embedded_input_3d, dtype=tf.float32)
#help(tf.nn.bidirectional_dynamic_rnn)

In [28]:
print(tf.shape(bidirect_encoder_states[0]))
print(tf.shape(bidirect_encoder_states[1]))

Tensor("Shape_5:0", shape=(3,), dtype=int32)
Tensor("Shape_6:0", shape=(3,), dtype=int32)


In [29]:
output_embedding_size = input_embedding_size

In [30]:
tf.__version__

'1.9.0-rc0'